In [ ]:
pip install MechanicalSoup

In [ ]:
pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from urllib.request import urlopen
import mechanicalsoup as ms
import re
import csv
import sys
sys.path.append('/content/drive/MyDrive/Projects/gran_turismo')
from car import Car

In [ ]:
def get_car_list(data):
  my_list = []
  for row in data:
    img = row.find('img')
    name = re.search("alt=.*src", str(img))
    name = name.group()
    name = name[5:len(name)-5]

    # Debugging
    # print("Currently on: ", name)

    cty = row.find('div', class_='cty')
    cty = re.sub("<.*?>", "", str(cty))

    # Price may not be defined for some cars
    # In such case, set the value to a negative number
    price = row.find('div', class_='price')
    price = re.sub("[^0-9]", "", str(price))
    try:
      price = int(price.replace(',', ''))
    except ValueError:
      price = -1.0

    tr = row.find('div', class_='tr')
    tr = re.sub("<.*?>", "", str(tr))

    aspi = row.find('div', class_='aspi')
    aspi = re.sub("<.*?>", "", str(aspi))

    car_sources = row.find('div', class_="car_source")
    car_sources = car_sources.find_all('b')
    for i in range(len(car_sources)):
      car_sources[i] = re.sub("<.*?>", "", str(car_sources[i]))

    car_tags = row.find('div', class_='car_tags')
    car_tags = car_tags.find_all('b')
    for i in range(len(car_tags)):
      car_tags[i] = re.sub("<.*?>", "", str(car_tags[i]))

    pp = row.find('span', class_='pp')
    pp = re.sub("<.*?>", "", str(pp))
    pp = float(pp[3:])

    tpw = row.find('span', class_='tpw')
    tpw = tpw.find_all('p')
    try:
      pw = float(re.sub("[^0-9]", "", str(tpw[0])))
    except ValueError:
      pw = -1.0

    try:
      weight = float(re.sub("[^0-9]", "", str(tpw[1])))
    except ValueError:
      weight = -1.0

    # 0-1000m may not be defined for some cars
    # In such case, set the value to a negative number
    zero_thou = 0.0
    try:
      zero_thou = re.search("0-1000m.*?</b>", str(row))
      zero_thou = float(re.sub("[^.0-9]", "", zero_thou.group()[6:]))
    except ValueError:
      zero_thou = -1.0

    # Grip may not be defined for some cars
    # In such case, set the value to a negative number
    grip = 0.0
    try:
      grip = re.search("Grip.*?</b>", str(row))
      grip = float(re.sub("[^.0-9]", "", grip.group()))
    except ValueError:
      grip = -1.0

    img_path = row.img['src']

    my_car = Car(name, cty, price, tr, aspi, car_sources, car_tags, pp, pw,
                weight, zero_thou, grip, img_path)
    my_list.append(my_car)

  return my_list

In [ ]:
browser = ms.Browser()

data = []

for i in range(0, 500, 50):
  url = "https://www.kudosprime.com/gt7/carlist.php"
  if i != 0:
    url = url + "?start=" + str(i)
  print(url)
  page = browser.get(url)
  result = page.soup.find_all("div", class_="car")
  data.extend(get_car_list(result))

https://www.kudosprime.com/gt7/carlist.php
https://www.kudosprime.com/gt7/carlist.php?start=50
https://www.kudosprime.com/gt7/carlist.php?start=100
https://www.kudosprime.com/gt7/carlist.php?start=150
https://www.kudosprime.com/gt7/carlist.php?start=200
https://www.kudosprime.com/gt7/carlist.php?start=250
https://www.kudosprime.com/gt7/carlist.php?start=300
https://www.kudosprime.com/gt7/carlist.php?start=350
https://www.kudosprime.com/gt7/carlist.php?start=400
https://www.kudosprime.com/gt7/carlist.php?start=450


In [ ]:
for car in data[0:10]:
  print(car)

name: Abarth 1500 Biposto Bertone B.A.T 1 '52, cty: No Gr, price: 1000000, tr: FR, aspi: NA, car sources: ['Legend Cars'], car tags: ['#Road Car'], pp: 264.0, pw: 73.0, weight: 1918.0, pwr: 26.273972602739725, 0-1000m: 35.75, grip: 0.85, img_path: images/cars/side/gt7_car_015.jpg?v=1
name: Abarth 500 '09, cty: No Gr, price: 29500, tr: FF, aspi: T, car sources: ['Brand Central', 'Used Cars', 'Reward'], car tags: ['#Road Car'], pp: 391.0, pw: 133.0, weight: 2282.0, pwr: 17.157894736842106, 0-1000m: 31.38, grip: 1.01, img_path: images/cars/side/gt7_car_016.jpg?v=2
name: Abarth 595 SS '70, cty: No Gr, price: 48900, tr: RR, aspi: NA, car sources: ['Used Cars'], car tags: ['#Road Car'], pp: 172.0, pw: 30.0, weight: 1036.0, pwr: 34.53333333333333, 0-1000m: 43.61, grip: 0.0, img_path: images/cars/side/gt7_car_017.jpg?v=2
name: Alfa Romeo 155 2.5 V6 TI '93, cty: Gr.4, price: 800000, tr: 4WD, aspi: NA, car sources: ['Legend Cars'], car tags: ['#Racing Car'], pp: 670.0, pw: 413.0, weight: 2227.0,

# 새 섹션

In [ ]:
with open('/content/drive/MyDrive/Projects/gran_turismo/gt_data.csv', 'w', encoding='UTF8', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(Car.get_head())
  for car in data:
    writer.writerow(car.to_list())